# Datos Finales

In [1]:
import pandas as pd
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [3]:
findf = pd.read_pickle('../data/interim/findf.pkl')
print('fin shape: ',findf.shape)

fin shape:  (554523, 7)


In [4]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada'],
      dtype='object')

In [5]:
findf.head()

,codigo_alumno,codigo_materia,anio_academico,nota,aprobado,promociono,año_cursada
0,27,850783,1993.0,10,10,NaN,NaN
1,27,850783,1993.0,2,NaN,NaN,NaN
2,27,881347,1994.0,7,7,NaN,NaN
3,27,952020,1997.0,4,4,NaN,NaN
4,27,952021,1997.0,5,5,NaN,NaN


# Criterios de limpieza

## Criterios 

basados en el análsis del dataset hecho en el notebook 02_Analisis_Datos_Alumnos.ipynb

* `codigo_alumno`
    - id
* `codigo_materia`
    - decision -> `usar variable`
    - no soportado
    - analizar manual o cambiarlo para que lo tome la libreria
* `anio_academico`
    - decisión -> `usar variable`
    - minimo 1972
* `nota`
    - decision -> `usar variable`
    - tiene mezclado numeros y 'aprobado' etc
        - hay que mejorar la variable para utilizarla -> hay otra variable `aprobado`. Hacer algo entre ambos.
        - tomar un criterio para esos casos
    - tiene missings
        - puede ser que sean ausentes
            - averiguar mejor analizando mas en profundidad los datos
            - adoptar criterio
* `aprobado`
    - decision -> `verificar`
    - tiene muchos missings
    - pareceria ser muy parecida a la variable `nota`
        - analizar bien la relacion entre ambas
        - quizas se pueda utilizar una sola
* `promociono`
    - decision -> `verificar` 
    - no soportada
        - analizar manualmente
* `año_cursada`
    - decision -> `usar variable pero verificar con el dataser de Cursadas`
    - minimo 1997
    - tiene 30% de missings

---
* `WARNING`
    - Tiene filas duplicadas
        - analizar bien que puede llegar a significar
            - si es un error -> borrarlas
            - intuision -> quizas si desaprobo o se ausento al final en el mismo año


# Analisis por variables

## Variable: `codigo_materia`

In [6]:
findf.codigo_materia.describe()

count     554523
unique      1099
top       950701
freq       22163
Name: codigo_materia, dtype: object

#### Conclusion

hay mas valores unicos que en el dataset de cursadas. Por lo que hay finales que no estan en cursadas. Por lo que nos puede decir que falta informacion en cursadas.

se esperaria que fuera alreves.


hacer:

* traer dataset de cursadas y ver cuantos alumnos tienen finales pero no cursadas de esas materias. evaluar eliminacion o analizar porque. Puede ser que sean equivalencias aunque me parece raro que sean tantas.


In [8]:
pd.DataFrame(findf.codigo_materia.unique()).to_csv('../data/interim/materias_fin_tmp.csv', index=False)

In [9]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada'],
      dtype='object')

## Variable: `anio_academico` y `año_cursada`

In [15]:
# pandas dataframe describe only for categorical variables
findf.describe(include=['object'])


,codigo_alumno,codigo_materia,nota,aprobado,promociono
count,554523,554523,414315,318318,116935
unique,24830,1099,16,10,8
top,79953,950701,2,7,8
freq,89,22163,78507,61221,44881


In [16]:
findf.describe()

,anio_academico,año_cursada
count,554522.000000,384831.000000
mean,2012.152851,2013.741985
std,5.934359,4.163146
min,1972.000000,1997.000000
25%,2008.000000,2010.000000
50%,2013.000000,2014.000000
75%,2017.000000,2017.000000
max,2022.000000,2023.000000


In [27]:
findf['anio_academico_str'] = findf['anio_academico'].astype(str)
findf['anio_cursada_str'] = findf['año_cursada'].astype(str)

aux = findf.groupby(['codigo_alumno', 'codigo_materia']).agg({'anio_academico_str': lambda x: x.nunique(), 'anio_cursada_str': lambda x: x.nunique()})
aux = aux.reset_index()

In [28]:
aux.columns = ['codigo_alumno', 'codigo_materia', 'anio_academico_str', 'anio_cursada_str']

In [30]:
aux[(aux.anio_cursada_str > 1)]

,codigo_alumno,codigo_materia,anio_academico_str,anio_cursada_str


#### conclusion

no hay distntos valores en anio cursada

##### preguntas

el anio_cursada coincide con el maximo de anio_academicos?

In [39]:
aux = findf.groupby(['codigo_alumno', 'codigo_materia']).agg({
    'anio_academico_str': 'max', 
    'anio_cursada_str': 'max',
    'anio_academico': 'max', 
    'año_cursada': 'max'
    })
aux = aux.reset_index()

In [42]:
aux['iguales'] = aux.anio_academico_str == aux.anio_cursada_str
aux2 = aux[(aux.iguales == False)&(aux.anio_cursada_str != 'nan')&((aux.anio_academico) - (aux.año_cursada) > 1)].copy()

In [43]:
aux2.codigo_alumno.nunique()

11064

#### conclusion

no se puede saber cual es la dif entre ambos años. Debido a que para un mismo alumno en todos los registros de finales es el mismo valor, se podria concluir cual es el año en que que se cursó la matria y aprobó. Se puede verificar.

In [45]:
curdf = pd.read_pickle('../data/interim/curdf.pkl')

In [46]:
curdf.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'modalidad',
       'turno', 'ciclo_lectivo', 'tipo_de_aprobacion',
       'cantidad_recursada_regular', 'descripción_recursada_regular',
       'cantidad_recursada_libre', 'descripción_de_recursada_libre'],
      dtype='object')

In [54]:
auxfin = findf[['codigo_alumno', 'codigo_materia', 'año_cursada']].drop_duplicates().copy()
# auxcur = curdf[['codigo_alumno', 'codigo_materia', 'ciclo_lectivo']].drop_duplicates().copy()
auxcur = curdf.groupby(['codigo_alumno', 'codigo_materia']).agg({'ciclo_lectivo':'max'}).reset_index().drop_duplicates().copy()

auxcurfin = pd.merge(auxfin, auxcur, how='left', on=['codigo_alumno', 'codigo_materia'])

In [55]:
auxcurfin.head()

,codigo_alumno,codigo_materia,año_cursada,ciclo_lectivo
0,27,850783,NaN,NaN
1,27,881347,NaN,NaN
2,27,952020,NaN,NaN
3,27,952021,NaN,NaN
4,27,952023,NaN,NaN


In [56]:
auxcurfin['iguales'] = auxcurfin.año_cursada == auxcurfin.ciclo_lectivo

In [60]:
auxcurfin[(auxcurfin.iguales == False)&(auxcurfin.ciclo_lectivo.notnull())&(auxcurfin.año_cursada.notnull())]

,codigo_alumno,codigo_materia,año_cursada,ciclo_lectivo,iguales


In [72]:
auxcurfin[(auxcurfin.iguales == False)]['ciclo_lectivo'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: ciclo_lectivo, dtype: float64

In [68]:
auxcurfin[(auxcurfin.iguales == False)]['año_cursada'].describe()

count    71675.000000
mean      2014.845790
std          3.537718
min       2006.000000
25%       2012.000000
50%       2015.000000
75%       2017.000000
max       2023.000000
Name: año_cursada, dtype: float64

In [58]:
auxcurfin.iguales.value_counts()

True     246259
False    209398
Name: iguales, dtype: int64

In [59]:
auxcurfin.shape

(455657, 5)

In [75]:
auxcurfin['NAN'] = auxcurfin['ciclo_lectivo'].isnull()

auxcurfin['NAN_iguales'] = auxcurfin['iguales'] == auxcurfin['NAN']
auxcurfin['NAN_iguales'].value_counts()

False    455657
Name: NAN_iguales, dtype: int64

#### conclusiones

año cursada del dataset de finales es igual al del ciclo lectivo maximo agrupando por alumno+materia en todos los casos donde el ciclo lectivo no es nulo

hay datos solo en año cursada, que no estan en ciclo lectivo, por lo que puede ser que no esté el dato en cursadas pero si esta en finales y se podria tomar de ahi.

In [76]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada', 'anio_academico_str', 'anio_cursada_str'],
      dtype='object')

## Variables: `nota` y `aprobado` y `promociono`

In [77]:
findf.nota.unique()

array(['10', '2', '7', '4', '5', '1', '6', '9', nan, '8', 'Aprobada', '3',
       'No aprobad', 'Acreditado', 'No Acredit', '0', 'No Suficie'],
      dtype=object)

In [78]:
findf.aprobado.unique()

array(['10', nan, '7', '4', '5', '6', '9', '8', 'Aprobada', 'No aprobad',
       'Acreditado'], dtype=object)

In [79]:
findf.promociono.unique()

array([nan, '7', '9', '8', '10', '6', '4', '5', 'Aprobada'], dtype=object)

In [80]:
findf.head(20)

,codigo_alumno,codigo_materia,anio_academico,nota,aprobado,promociono,año_cursada,anio_academico_str,anio_cursada_str
0,27,850783,1993.0,10,10,NaN,NaN,1993.0,nan
1,27,850783,1993.0,2,NaN,NaN,NaN,1993.0,nan
2,27,881347,1994.0,7,7,NaN,NaN,1994.0,nan
3,27,952020,1997.0,4,4,NaN,NaN,1997.0,nan
4,27,952021,1997.0,5,5,NaN,NaN,1997.0,nan
5,27,952023,1997.0,7,7,NaN,NaN,1997.0,nan
6,27,950611,1998.0,4,4,NaN,NaN,1998.0,nan
7,27,950701,1998.0,1,NaN,NaN,NaN,1998.0,nan
8,27,950702,1998.0,4,4,NaN,NaN,1998.0,nan
9,27,951420,1998.0,7,7,NaN,NaN,1998.0,nan


In [81]:
findf_notas = findf.copy()
findf_notas['nota_aprobo'] = findf.nota == findf.aprobado
findf_notas['nota_promociono'] = findf.nota == findf.promociono

In [83]:
findf_notas.head()

,codigo_alumno,codigo_materia,anio_academico,nota,aprobado,promociono,año_cursada,anio_academico_str,anio_cursada_str,nota_aprobo,nota_promociono
0,27,850783,1993.0,10,10,NaN,NaN,1993.0,nan,True,False
1,27,850783,1993.0,2,NaN,NaN,NaN,1993.0,nan,False,False
2,27,881347,1994.0,7,7,NaN,NaN,1994.0,nan,True,False
3,27,952020,1997.0,4,4,NaN,NaN,1997.0,nan,True,False
4,27,952021,1997.0,5,5,NaN,NaN,1997.0,nan,True,False


In [84]:
findf_notas[['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']].head(20)

,nota,aprobado,promociono,nota_aprobo,nota_promociono
0,10,10,NaN,True,False
1,2,NaN,NaN,False,False
2,7,7,NaN,True,False
3,4,4,NaN,True,False
4,5,5,NaN,True,False
5,7,7,NaN,True,False
6,4,4,NaN,True,False
7,1,NaN,NaN,False,False
8,4,4,NaN,True,False
9,7,7,NaN,True,False


In [87]:
findf_notas[(findf_notas.nota_aprobo == False)|(findf_notas.nota_promociono == False)][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono
0,10,10,NaN,True,False
1,2,NaN,NaN,False,False
2,7,7,NaN,True,False
3,4,4,NaN,True,False
4,5,5,NaN,True,False
...,...,...,...,...,...
554518,7,7,NaN,True,False
554519,8,8,NaN,True,False
554520,10,10,NaN,True,False
554521,9,9,NaN,True,False


In [89]:
findf_notas[~((findf_notas.nota_aprobo == True)|(findf_notas.nota_promociono == True))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono
1,2,NaN,NaN,False,False
7,1,NaN,NaN,False,False
12,2,NaN,NaN,False,False
17,1,NaN,NaN,False,False
24,2,NaN,NaN,False,False
...,...,...,...,...,...
554496,2,NaN,NaN,False,False
554504,NaN,NaN,8,False,False
554508,2,NaN,NaN,False,False
554511,2,NaN,NaN,False,False


In [90]:
findf_notas[((findf_notas.nota_aprobo == True)&(findf_notas.nota_promociono == True))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono


In [91]:
findf_notas[((findf_notas.nota_aprobo == False))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono
1,2,NaN,NaN,False,False
7,1,NaN,NaN,False,False
12,2,NaN,NaN,False,False
17,1,NaN,NaN,False,False
24,2,NaN,NaN,False,False
...,...,...,...,...,...
554496,2,NaN,NaN,False,False
554504,NaN,NaN,8,False,False
554508,2,NaN,NaN,False,False
554511,2,NaN,NaN,False,False


In [92]:
findf_notas[((findf_notas.nota.isnull())&findf_notas.aprobado.isnull()&findf_notas.promociono.isnull())][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono
11319,NaN,NaN,NaN,False,False
11840,NaN,NaN,NaN,False,False
12850,NaN,NaN,NaN,False,False
14420,NaN,NaN,NaN,False,False
18707,NaN,NaN,NaN,False,False
...,...,...,...,...,...
554446,NaN,NaN,NaN,False,False
554448,NaN,NaN,NaN,False,False
554452,NaN,NaN,NaN,False,False
554457,NaN,NaN,NaN,False,False


In [93]:
# nota distinto a aprobo y que aprobo no es nan
findf_notas[((findf_notas.nota != findf_notas.aprobado)&(findf_notas.aprobado.notnull()))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono


In [95]:
findf_notas[((findf_notas.nota != findf_notas.promociono)&(findf_notas.promociono.notnull()))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono
26,NaN,NaN,7,False,False
32,NaN,NaN,9,False,False
64,NaN,NaN,7,False,False
151,NaN,NaN,7,False,False
156,NaN,NaN,7,False,False
...,...,...,...,...,...
554477,NaN,NaN,9,False,False
554480,NaN,NaN,10,False,False
554484,NaN,NaN,7,False,False
554486,NaN,NaN,9,False,False


In [98]:
findf_notas[(((findf_notas.nota != findf_notas.promociono)&(findf_notas.nota.notnull()))&
             ((findf_notas.nota != findf_notas.promociono)&(findf_notas.promociono.notnull())))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono


In [99]:
findf_notas[(((findf_notas.nota != findf_notas.aprobado)&(findf_notas.nota.notnull()))&
             ((findf_notas.nota != findf_notas.aprobado)&(findf_notas.aprobado.notnull())))][['nota', 'aprobado', 'promociono', 'nota_aprobo', 'nota_promociono']]

,nota,aprobado,promociono,nota_aprobo,nota_promociono


#### conclusiones

* `nota` son todas las notas pero no están las de `promocion`

* `aprobado` tiene las notas de `nota` excepto las notas de desaprobación numéricas, pero si tiene las notas de desaprobación de texto como 'No aprobado' 

* `promociono` tiene las notas de promoción numérica y 'Aprobado' y no esta ni en las variables `nota` ni `aprobado`

`criterios`: 

1. si en los 3 es 'NAN' se tomará como ausente

2. transformar las columnas de la siguiente manera:

2.a. tipo_aprobacion: aprobacion, acreditacion, promocion, ausente

2.b. nota_numerica: solo números. Si no hay número pero si hay 'aprobacion' o 'acreditacion' se tomará la nota mínima de aprobacion (4 o 7 porque cambió la reglamentación)

2.c. nota_cualitativa: aprobado, No aprobado, ausente, acreditado, no acreditado



In [102]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada', 'anio_academico_str', 'anio_cursada_str'],
      dtype='object')

In [106]:
findf.nota.unique()

array(['10', '2', '7', '4', '5', '1', '6', '9', nan, '8', 'Aprobada', '3',
       'No aprobad', 'Acreditado', 'No Acredit', '0', 'No Suficie'],
      dtype=object)

In [109]:
findf['tipo_aprobacion'] = np.where(
                                (findf.nota == 'Acreditado')|(findf.nota == 'No Acredit'), 'Acreditacion',
                                        np.where(findf.nota.notnull(), 'Aprobacion',
                                                np.where(findf.promociono.notnull(), 'promocion', 'ausente')
                                                )
                                )

In [110]:
findf.tipo_aprobacion.unique()

array(['Aprobacion', 'promocion', 'Acreditacion', 'ausente'], dtype=object)

In [111]:
findf['nota_comb'] = findf.nota.combine_first(findf.aprobado).combine_first(findf.promociono)
findf['nota_comb']

0         10
1          2
2          7
3          4
4          5
          ..
554518     7
554519     8
554520    10
554521     9
554522     8
Name: nota_comb, Length: 554523, dtype: object

In [112]:
findf.nota_comb.unique()

array(['10', '2', '7', '4', '5', '1', '6', '9', '8', 'Aprobada', '3',
       'No aprobad', 'Acreditado', nan, 'No Acredit', '0', 'No Suficie'],
      dtype=object)

In [113]:
num_map = {'10':10, '9':9, '8':8, '7':7, '6':6, '5':5, '4':4, '3':3, '2':2, '1':1, '0':0}
cual_map = {'Aprobada': 'Aprobada', 'No aprobad': 'No Aprobada', 'Acreditado': 'Acreditada', 'No Acredit': 'No Acreditada', 'No Suficie': 'No Aprobada'}

findf['nota_num'] = findf.nota_comb.map(num_map)
findf['nota_cual'] = findf.nota_comb.map(cual_map)

In [118]:
findf.nota_num.isnull().sum()

29729

In [119]:
findf.nota_cual.isnull().sum()

548067

In [121]:
findf.nota_num[0]

10.0

In [122]:

# tener en cuenta el cambio de nota de un año a otro porque hay que convertir las notas para que sean comparables
findf['nota_txt'] = np.where(findf.nota_cual.notnull(), findf.nota_cual, 
                            np.where(findf.nota_num >= 4, 'Aprobada', 'No Aprobada')
                            )

#### pendientes

hacer la transformacion de las notas numericas para igual las escalas luego de la reglamentacion que pasaron de 4 a 7 las aporbaciones d elos examenes


In [123]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota', 'aprobado',
       'promociono', 'año_cursada', 'anio_academico_str', 'anio_cursada_str',
       'tipo_aprobacion', 'nota_comb', 'nota_num', 'nota_cual', 'nota_txt'],
      dtype='object')

In [124]:
COLS = ['codigo_alumno', 'codigo_materia', 'anio_academico', 'año_cursada', 'tipo_aprobacion', 'nota_num', 'nota_cual']

findf[COLS].head()

,codigo_alumno,codigo_materia,anio_academico,año_cursada,tipo_aprobacion,nota_num,nota_cual
0,27,850783,1993.0,NaN,Aprobacion,10.0,NaN
1,27,850783,1993.0,NaN,Aprobacion,2.0,NaN
2,27,881347,1994.0,NaN,Aprobacion,7.0,NaN
3,27,952020,1997.0,NaN,Aprobacion,4.0,NaN
4,27,952021,1997.0,NaN,Aprobacion,5.0,NaN


In [125]:
findf.shape

(554523, 14)

In [126]:
findf.to_pickle('../data/interim/findf_01.pkl')